In [2]:
%pip install --upgrade --quiet langchain langchain-community langchain-ollama
%pip install --upgrade --quiet pypdf pandas streamlit python-dotenv
%pip install opencv-python python-pptx


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# from langchain_community.document_loaders import OllamaPDFLoader
# from langchain_community.document_loaders import OllamaTextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from pptx import Presentation

import cv2
import pytesseract
import os
import pandas as pd
import streamlit as st
from dotenv import load_dotenv

c:\Users\tonyj\OneDrive\Desktop\Sem8\Thesis\RAG_project\myenv\Lib\site-packages\IPython\core\interactiveshell.py:3670: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
load_dotenv()

True

In [6]:
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY")
if OLLAMA_API_KEY is None:
    raise ValueError("Please set the OLLAMA_API_KEY environment variable.")

# OLLAMA_API_KEY

## Define Our LLM

In [7]:
llm = ChatOllama(model="llama3.2", temperature=0.1, api_key=OLLAMA_API_KEY)

## Process PDF Document

### Load the PDF file

In [8]:


def load_file(path):
    text_runs = []
    if path.endswith(".pdf"):
        loader = PyPDFLoader(path)
        pages = loader.load()
        print(pages)
        return pages
    elif path.endswith(".pptx"):
        prs = Presentation(path)
        for slide in prs.slides:
            for shape in slide.shapes:
                if not shape.has_text_frame:
                    continue
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        text_runs.append(run.text)
                        print(text_runs)
        return text_runs
    elif path.endswith('.jpg') or path.endswith('.jpeg') or path.endswith('.png'):
            image = cv2.imread(path)
            if image is not None:
                text = pytesseract.image_to_string(image)
                print(text)
            return text
    else:
        print("Incorrect file format, only supported file formats are pdf, jpg, jpeg, pptx, png.")
        
text = load_file("data/Bhavya Kajani-1.pdf")
print("FIle PRINTED ABOVE")


[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-03-15T19:52:06+01:00', 'author': 'Bhavya Kajani', 'moddate': '2025-02-17T15:29:39+01:00', 'source': 'data/Bhavya Kajani-1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Bhavya Kajani \n+48 729 360 482 | Email | LinkedIn  | GitHub  \n Experience\nAutomation Tester - EY (GDS), Warsaw | July 2024 - Present\n• Creating Test Framework for API testing using Java, Junit5 and RestAssured for RESTAPI and\n       performed UI testing using Java, Selenium WebDriver and TestNg for parallel test case execution.\n• Performing manual tests to ensure the quality of the web applications like, exploratory, regression, and\n                testing the data consistency while providing suggestions and making test cases to increase the \n software quality\n         •   Reported bugs to log the issues in Azure DevOps for proper documentation and know

### Split the document into chunks - Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
    length_function = len,
    separators = ["\n\n", "\n", " "],
)
chunks_without_split = text # to compare with the split chunks
chunks = text_splitter.split_documents(text)

### Create embeddings and vector store

In [10]:
## Note: Look up text embedding 
def get_embeddings_function():
    embeddings = OllamaEmbeddings(
        model="llama3.2",
    )
    return embeddings

embeddings = get_embeddings_function()
test_vector = embeddings.embed_documents([chunks[0].page_content])

### Create Vector database

In [11]:
# Create a Chroma database from the documents
import uuid

def create_vectorstore(chunks, embeddings, vectorestore_path):

    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    unique_ids = set()
    unique_chunks = []

    for chunk, id in zip(chunks, ids):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)
    # Create a Chroma database from the documents
    vectorestore = Chroma.from_documents(
        documents=unique_chunks,
        ids=list(unique_ids),
        embedding=embeddings,
        persist_directory=vectorestore_path)

    # Persist the vector store to disk
    vectorestore.persist()

    return vectorestore


In [12]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
vectorestore = create_vectorstore(chunks=chunks, embeddings=embeddings, vectorestore_path="vectorstore_chroma_db")

C:\Users\tonyj\AppData\Local\Temp\ipykernel_8116\2058071402.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorestore.persist()


## Query for relevant data

In [14]:
vectorestore = Chroma(persist_directory="vectorstore_chroma_db", embedding_function=embeddings)

C:\Users\tonyj\AppData\Local\Temp\ipykernel_8116\1665830820.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorestore = Chroma(persist_directory="vectorstore_chroma_db", embedding_function=embeddings)


In [15]:
retriever = vectorestore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
relevant_chunk = retriever.invoke(" ")
relevant_chunk

[Document(metadata={'page_label': '1', 'total_pages': 1, 'page': 0, 'source': 'data/Bhavya Kajani-1.pdf', 'creator': 'Microsoft® Word for Microsoft 365', 'producer': 'Microsoft® Word for Microsoft 365', 'moddate': '2025-02-17T15:29:39+01:00', 'author': 'Bhavya Kajani', 'creationdate': '2024-03-15T19:52:06+01:00'}, page_content='Bhavya Kajani \n+48 729 360 482 | Email | LinkedIn  | GitHub  \n Experience\nAutomation Tester - EY (GDS), Warsaw | July 2024 - Present\n• Creating Test Framework for API testing using Java, Junit5 and RestAssured for RESTAPI and\n       performed UI testing using Java, Selenium WebDriver and TestNg for parallel test case execution.\n• Performing manual tests to ensure the quality of the web applications like, exploratory, regression, and\n                testing the data consistency while providing suggestions and making test cases to increase the \n software quality\n         •   Reported bugs to log the issues in Azure DevOps for proper documentation and know

### Prompt Template

In [16]:
PROMPT_TEMPLATE = """
Extract the following from this resume in natural language:
    - Skills: [list]
    - Experience: [years, roles]
    - Education: [degree, university]
    - Name: [name]
    - Contact: [contact_info]
    Text: {context}
    - Do not make up any information.    
    - Answer in JSON format. No further explanation nor natural language outside of JSON can be present. 
    - Do not include any other information.
    - Give the output without any escape characters and only in JSON format.
    """


In [17]:
# Create a context from the relevant chunks
context_text = "\n\n----\n\n".join([doc.page_content for doc in relevant_chunk])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text)

print(prompt)

Human: 
Extract the following from this resume in natural language:
    - Skills: [list]
    - Experience: [years, roles]
    - Education: [degree, university]
    - Name: [name]
    - Contact: [contact_info]
    Text: Bhavya Kajani 
+48 729 360 482 | Email | LinkedIn  | GitHub  
 Experience
Automation Tester - EY (GDS), Warsaw | July 2024 - Present
• Creating Test Framework for API testing using Java, Junit5 and RestAssured for RESTAPI and
       performed UI testing using Java, Selenium WebDriver and TestNg for parallel test case execution.
• Performing manual tests to ensure the quality of the web applications like, exploratory, regression, and
                testing the data consistency while providing suggestions and making test cases to increase the 
 software quality
         •   Reported bugs to log the issues in Azure DevOps for proper documentation and knowledge of scrum 
 framework including participation in scrum rituals. 
Software Tester - Lionbridge Technologies, Warsaw 

### Generate Responses

In [25]:
%pip install json-repair

json = llm.invoke(prompt).model_dump_json()
import json as js
from json_repair import repair_json

print(js.dumps(json, indent=4))
print(repair_json(json))  # Check if the key "skills" exists in the JSON response


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
"{\"content\":\"{\\\"name\\\": \\\"Bhavya Kajani\\\", \\\"contact_info\\\": \\\"+48 729 360 482 | Email | LinkedIn | GitHub\\\", \\\"education\\\": \\\"Warsaw University of Technology, Warsaw Bachelor\u2019s in Computer Science 2021 \u2013 Present\\\", \\\"experience\\\": \\\"[7, \\\\\\\"Automation Tester - EY (GDS), Warsaw\\\\\\\", \\\\\\\"Software Tester - Lionbridge Technologies, Warsaw\\\\\\\"]\\\", \\\"skills\\\": \\\"[\\\\\\\"Java\\\\\\\", \\\\\\\"C/C++\\\\\\\", \\\\\\\"Selenium\\\\\\\", \\\\\\\"RestAssured\\\\\\\", \\\\\\\"Junit\\\\\\\", \\\\\\\"TestNg\\\\\\\", \\\\\\\"VSCode\\\\\\\", \\\\\\\"IntelliJ\\\\\\\", \\\\\\\"Postman\\\\\\\", \\\\\\\"SwaggerUI\\\\\\\", \\\\\\\"Azure DevOps\\\\\\\"]\\\"}}\",\"additional_kwargs\":{},\"response_metadata\":{\"model\":\"llama3.2\",\"created_at\":\"2025-05-30T18:17:27.1014597Z\",\"done\":true,\"done_reason\":\"stop\",\"total_duration\":5228431800,\"load_duration\":29482600,\"pr

#### Method 2: Using LangChain Expression Language to extract the information from the JSON response

In [19]:
def format_docs(docs):
    return "\n\n----\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question":RunnablePassthrough()}
            | llm 
            | prompt_template,
)




### Structure Response

In [20]:
class ResumeParser(BaseModel):
    """Extracted information from a resume."""
    skills: list[str] = Field(description="List of skills")
    experience: list[str] = Field(description="List of experience")
    education: list[str] = Field(description="List of education")
    name: str = Field(description="Name of the person")
    contact_info: str = Field(description="Contact information")


In [23]:
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt_template
    | llm.with_structured_output(ResumeParser, method='json_schema')
    
)
print(rag_chain)
print(rag_chain.model_dump_json)

first={
  context: VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001E2CF527890>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
} middle=[ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nExtract the following from this resume in natural language:\n    - Skills: [list]\n    - Experience: [years, roles]\n    - Education: [degree, university]\n    - Name: [name]\n    - Contact: [contact_info]\n    Text: {context}\n    - Do not make up any information.    \n    - Answer in JSON format. No further explanation nor natural language outside of JSON can be present. \n    - Do not include any other information.\n    - Give the output without any escape characters and only in JSON format

### Transform the output to a pandas dataframe

In [24]:
structred_response = rag_chain.invoke("")
print(structred_response)
print(structred_response.skills)
df = pd.DataFrame([structred_response])
df.head()
print(df.to_json(orient="records", indent=2))


skills=['Java', 'C/C++', 'Selenium', 'RestAssured', 'Junit', 'TestNg'] experience=['Automation Tester - EY (GDS), Warsaw | July 2024 - Present', 'Software Tester - Lionbridge Technologies, Warsaw | July 2023 – January 2024'] education=['Warsaw University of Technology, Warsaw | Bachelor’s in Computer Science | 2021 – Present', 'Sarvajanik School Of Science, India | High School | 2017-2019', 'Metas Adventist School, India | School | 2006-2017'] name='Bhavya Kajani' contact_info='+48 729 360 482 | Email | LinkedIn | GitHub'
['Java', 'C/C++', 'Selenium', 'RestAssured', 'Junit', 'TestNg']
[
  {
    "0":[
      "skills",
      [
        "Java",
        "C\/C++",
        "Selenium",
        "RestAssured",
        "Junit",
        "TestNg"
      ]
    ],
    "1":[
      "experience",
      [
        "Automation Tester - EY (GDS), Warsaw | July 2024 - Present",
        "Software Tester - Lionbridge Technologies, Warsaw | July 2023 \u2013 January 2024"
      ]
    ],
    "2":[
      "education"